# Summarize Assignment of On-board Survey Demand
This notebook uses the Acceptance Criteria infrastructure to summarize the results of the assignment of the on-board survey demand to the TM2 networks. 

In [1]:
from tm2py.acceptance.canonical import Canonical
from tm2py.acceptance.simulated import Simulated
from tm2py.acceptance.observed import Observed
from tm2py.acceptance.acceptance import Acceptance

import pandas as pd

Inro not found. Skipping inro setup.


### Remote I/O

In [2]:
scenario_config_file = "../examples/scenario_config.toml"
observed_config_file = "../examples/observed_data.toml"
canonical_crosswalk_file = "../examples/canonical_crosswalk.toml"

input_previous_summary_filename = "on-board-assignment-summary.csv"

output_folder_root = "/Users/wsp/Documents/GitHub/tm2py/examples/temp_on_board_assign/"
output_filename = "on-board-assignment-summary.csv"

## Parameters

In [3]:
append_or_create = "create" 
iteration_description = "Uncogested with Link21 Parameters"
iteration_number = 0

In [4]:
c = Canonical(canonical_crosswalk_file, on_board_assign_summary=True)

In [5]:
s = Simulated(canonical=c, scenario_file=scenario_config_file, model_file=None, on_board_assign_summary=True)
s.reduce_on_board_assignment_boardings(time_period_list=["am"])

In [6]:
s.simulated_boardings_df.head()

,daily_line_name,tm2_mode,line_mode,operator,technology,fare_system,total_boarding,total_hour_cap,time_period,line_name
0,10_573_198,12,o,Emery Go-Round,Local bus,18,0.0,33.750000,daily,N/A -- Daily Record
1,10_573_199,12,o,Emery Go-Round,Local bus,18,0.0,207.692308,daily,N/A -- Daily Record
2,10_573_200,12,o,Emery Go-Round,Local bus,18,0.0,207.692308,daily,N/A -- Daily Record
3,10_573_201,12,o,Emery Go-Round,Local bus,18,0.0,33.750000,daily,N/A -- Daily Record
4,10_573_202,12,o,Emery Go-Round,Local bus,18,0.0,192.857143,daily,N/A -- Daily Record


In [7]:
o = Observed(canonical=c, observed_file=observed_config_file, on_board_assign_summary=True)
o.reduce_on_board_survey(read_file_from_disk=False)

In [8]:
df = o.reduced_transit_on_board_df.copy()
df.head()
# df = df[df["time_period"] == "am"].copy()
df = df[df["survey_route"] == "MUNI___N Judah"].copy()
df


,survey_tech,survey_operator,survey_route,survey_boardings,time_period,florida_threshold,survey_agency,standard_route_id,daily_line_name,canonical_operator,standard_route_short_name,standard_route_long_name,standard_line_name,standard_headsign
108,light rail,SF Muni,MUNI___N Judah,47160.718410,daily,0.25,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,N/A -- Daily Record,N/A -- Daily Record
1108,light rail,SF Muni,MUNI___N Judah,6051.687860,am,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_AM_d0_s1126,Ocean Beach via Downtown
1109,light rail,SF Muni,MUNI___N Judah,6051.687860,am,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_AM_d1_s1134,Caltrain via Downtown
1110,light rail,SF Muni,MUNI___N Judah,697.354685,ea,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_EA_d0_s1127,Ocean Beach via Downtown
1111,light rail,SF Muni,MUNI___N Judah,697.354685,ea,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_EA_d1_s1134,Caltrain via Downtown
1112,light rail,SF Muni,MUNI___N Judah,2730.224655,ev,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_EV_d0_s1124,Ocean Beach
1113,light rail,SF Muni,MUNI___N Judah,2730.224655,ev,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_EV_d1_s1132,Duboce & Church
1114,light rail,SF Muni,MUNI___N Judah,5975.938215,md,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_MD_d0_s1126,Ocean Beach via Downtown
1115,light rail,SF Muni,MUNI___N Judah,5975.938215,md,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_MD_d1_s1134,Caltrain via Downtown
1116,light rail,SF Muni,MUNI___N Judah,8125.153790,pm,NaN,SF Muni,368,22_SFMTA_368,SF Muni,N,JUDAH,22_SFMTA_368_PM_d0_s1126,Ocean Beach via Downtown


In [8]:
a = Acceptance(canonical=c, simulated=s, observed=o, output_file_root=output_folder_root)  
compare_df = a.compare_transit_boardings()

In [9]:
compare_df.head()

,tm2_mode,line_mode,operator,technology,time_period,total_boarding,survey_route,survey_boardings,florida_threshold,daily_line_name,...,line_name,survey_tech,survey_operator,survey_agency,standard_route_id,canonical_operator,standard_route_short_name,standard_route_long_name,standard_line_name,standard_headsign
0,14.0,o,Caltrain,Local Bus,am,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.0,o,Caltrain,Local Bus,daily,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120.0,h,BART,Heavy Rail,am,226163.591854,BART,134161.945802,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120.0,h,BART,Heavy Rail,daily,226163.591854,BART,431650.949580,0.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130.0,i,Caltrain,Commuter Rail,am,0.000000,Caltrain,20978.587956,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Make Output

In [10]:
compare_df["iteration_description"] = iteration_description
compare_df["iteration_number"] = iteration_number

if append_or_create == "create":
    compare_df.to_csv(output_folder_root + output_filename, index=False)
else:
    previous_df = pd.read_csv(output_folder_root + input_previous_summary_filename, dtype=compare_df.dtypes.to_dict())
    pd.concat([previous_df, compare_df], ignore_index=True).to_csv(output_folder_root + output_filename, index=False)